In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
import pickle

# Load the model, scaler, and choices dictionary
with open('training/model.pkl', 'rb') as f: 
    model = pickle.load(f)
with open('training/scaler.pkl', 'rb') as f: 
    scaler = pickle.load(f)
with open('training/choices.pkl', 'rb') as f: 
    choices = pickle.load(f)

# Define the categorical and continuous columns
cat_cols = [col for col in choices.keys() if choices[col] is not None]
cont_cols = [col for col in choices.keys() if choices[col] is None]

# Function to predict loan approval
def predict(input_data): 
    """
    Helper function to make predictions based on input data.
    """
    # Create a DataFrame with the correct columns
    X = pd.DataFrame(columns=choices.keys())
    
    # Map input data to the correct columns
    X.loc[0, 'ApplicantIncome'] = input_data['Combined_income'] * 0.6  # Example split
    X.loc[0, 'CoapplicantIncome'] = input_data['Combined_income'] * 0.4  # Example split
    X.loc[0, 'LoanAmount'] = input_data['Requested_amount']
    X.loc[0, 'Loan_Amount_Term'] = 360  # Example value, adjust as necessary
    
    # Handle categorical variables
    X.loc[0, 'Credit_History'] = 1 if input_data['Credit_history'] == 'Yes' else 0
    X.loc[0, 'Dependents_0'] = 1 if input_data['Dependents'] == '0' else 0
    X.loc[0, 'Dependents_1'] = 1 if input_data['Dependents'] == '1' else 0
    X.loc[0, 'Dependents_2'] = 1 if input_data['Dependents'] == '2' else 0
    X.loc[0, 'Dependents_3+'] = 1 if input_data['Dependents'] == '3+' else 0
    X.loc[0, 'Education_Graduate'] = 0 if input_data['College_degree'] == 'No' else 1
    X.loc[0, 'Education_Not Graduate'] = 1 if input_data['College_degree'] == 'No' else 0
    X.loc[0, 'Property_Area_Urban'] = 1 if input_data['Community_type'] == 'Urban' else 0
    X.loc[0, 'Property_Area_Semiurban'] = 1 if input_data['Community_type'] == 'Suburban' else 0
    X.loc[0, 'Property_Area_Rural'] = 1 if input_data['Community_type'] == 'Rural' else 0
    
    # Ensure all columns are present
    X = X.fillna(0)
    
    # Debugging statements to inspect the data
    print("Input DataFrame:")
    print(X.head())
    
    # Scale input data
    X_transformed = scaler.transform(X)
    print("Transformed Features:")
    print(X_transformed)
    
    # Make prediction
    output = model.predict(X_transformed)
    print("Model Output:")
    print(output)
    
    # Apply threshold to determine binary class
    threshold = 0.5
    prediction = (output[0] > threshold).astype(int)
    
    # Return 'No' for 0 and 'Yes' for 1
    return 'Yes' if prediction == 1 else 'No'

# Define a set of test inputs
test_inputs = [
    # Input where we expect 'No'
    {'Combined_income': 5000, 'Credit_history': 'No', 'Requested_amount': 200, 'Community_type': 'Urban', 'Dependents': '0', 'College_degree': 'Yes'},
    
    # Input where we expect 'Yes'
    {'Combined_income': 5000, 'Credit_history': 'Yes', 'Requested_amount': 200, 'Community_type': 'Urban', 'Dependents': '1', 'College_degree': 'No'},
    
    # Another input where we might expect 'No'
    {'Combined_income': 1500, 'Credit_history': 'No', 'Requested_amount': 100, 'Community_type': 'Rural', 'Dependents': '2', 'College_degree': 'No'},
    
    # Another input where we might expect 'Yes'
    {'Combined_income': 7000, 'Credit_history': 'Yes', 'Requested_amount': 300, 'Community_type': 'Suburban', 'Dependents': '3+', 'College_degree': 'Yes'},
    
    # Edge case input
    {'Combined_income': 0, 'Credit_history': 'No', 'Requested_amount': 0, 'Community_type': 'Urban', 'Dependents': '0', 'College_degree': 'No'}
]

# Run the prediction function with each test input and print results
for i, input_data in enumerate(test_inputs):
    print(f"Test Case {i + 1}:")
    result = predict(input_data)
    print("Prediction Result:")
    print(result)
    print("-" * 40)


Test Case 1:
Input DataFrame:
   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0           3000.0             2000.0         200               360   

   Credit_History  Gender_Female  Gender_Male  Married_No  Married_Yes  \
0               0              0            0           0            0   

   Dependents_0  Dependents_1  Dependents_2  Dependents_3+  \
0             1             0             0              0   

   Education_Graduate  Education_Not Graduate  Self_Employed_No  \
0                   1                       0                 0   

   Self_Employed_Yes  Property_Area_Rural  Property_Area_Semiurban  \
0                  0                    0                        0   

   Property_Area_Urban  
0                    1  
Transformed Features:
[[-0.39166957  0.13683214  0.5868408   0.29768245 -2.37938872 -0.45429969
  -2.06871213 -0.70171306 -1.4120561   0.90472013 -0.46405445 -0.45103367
  -0.30184617  0.50889523 -0.50889523 -2.02844357 -0.4145

/var/folders/jn/wpjc41457v19x74lyfk09mlm0000gn/T/ipykernel_54424/3158420084.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X = X.fillna(0)
/var/folders/jn/wpjc41457v19x74lyfk09mlm0000gn/T/ipykernel_54424/3158420084.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X = X.fillna(0)
/var/folders/jn/wpjc41457v19x74lyfk09mlm0000gn/T/ipykernel_54424/3158420084.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to th

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Model Output:
[[0.8678772]]
Prediction Result:
Yes
----------------------------------------
Test Case 5:
Input DataFrame:
   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0              0.0                0.0           0               360   

   Credit_History  Gender_Female  Gender_Male  Married_No  Married_Yes  \
0               0              0            0           0            0   

   Dependents_0  Dependents_1  Dependents_2  Dependents_3+  \
0             1             0             0              0   

   Education_Graduate  Education_Not Graduate  Self_Employed_No  \
0                   0                       1                 0   

   Self_Employed_Yes  Property_Area_Rural  Property_Area_Semiurban  \
0                  0                    0                        0   

   Property_Area_Urban  
0                    1  
Transformed Features:
[[-0.85586261 -0.52115617 -1.64925501  0.29768245 -2.37938872 -0.45429969
 

/var/folders/jn/wpjc41457v19x74lyfk09mlm0000gn/T/ipykernel_54424/3158420084.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X = X.fillna(0)
